# Transfer learning & fine-tuning
by uramoon@kw.ac.kr<br><br>

Xception을 사용하여 고양이와 개를 분류해봅시다.<br>
런타임 유형은 가급적 GPU로 설정하세요. (CPU도 가능)<br><br>

**Author:** [fchollet](https://twitter.com/fchollet)<br>
**Date created:** 2020/04/15<br>
**Last modified:** 2020/05/12<br>
**Description:** Complete guide to transfer learning & fine-tuning in Keras.<br>
(<a href="https://raw.githubusercontent.com/ronreiter/interactive-tutorials/master/LICENSE">Apache 2.0 License</a>)

## Setup

In [1]:
import numpy as np
import tensorflow as tf
from tensorflow import keras

## 소개

**전이 학습**은 어떤 문제에 대해 학습된 특징들을 새로운 문제의 풀이에 사용하는 것을 뜻합니다. <br>예를 들면, 라쿤을 인식하는 모델에서 사용되는 특징들로 너구리를 인식하는 데 사용할 수 있을 것입니다.<br>

<img src="https://extension.umd.edu/sites/extension.umd.edu/files/styles/optimized/public/2021-02/hgic_veg_wildlife_raccoon.jpg?itok=p4k_Z_CF" height="200"><figcaption>라쿤 사진 출처: https://extension.umd.edu/resource/raccoons</figcaption>

<img src="https://image-notepet.akamaized.net/resize/620x-/seimage/20171108%2Fe6d1ec360a4ab04e21e580882d9c989e.jpg" height="200"><figcaption>너구리 사진 출처: https://www.notepet.co.kr/news/article/article_view/?idx=10434</figcaption>

전이학습은 다음의 수행절차를 갖습니다.
1. 기존에 훈련된 모델을 가져온다.
2. 가져온 모델을 훈련이 불가능하도록 설정한다. (가중치들을 고정시킴)
3. 가져온 모델에 몇 개의 층을 추가한다.
4. 내가 추가한 층만 나의 데이터로 훈련시킨다.

추가적으로 고정시킨 모델을 훈련 가능하도록 설정해 세부 튜닝을 시도할 수도 있습니다.

This is adapted from
[Deep Learning with Python](https://www.manning.com/books/deep-learning-with-python)
 and the 2016 blog post
["building powerful image classification models using very little
 data"](https://blog.keras.io/building-powerful-image-classification-models-using-very-little-data.html).

## 데이터 가져오기
원래 25,000장의 그림이 있는데 15,000장만 가져와서 그 중 10,000장은 훈련, 2500장은 검증, 2500장은 테스트에 사용하겠습니다.<br>
(오염된 이미지들이 있어서 다 받아오진 못하고 수 백장의 사진이 걸러집니다.)

In [2]:
import tensorflow_datasets as tfds

tfds.disable_progress_bar()

# 임시 코드 (다운로드가 안되면 지우세요.)
setattr(tfds.image_classification.cats_vs_dogs, '_URL',"https://download.microsoft.com/download/3/E/1/3E1C3F21-ECDB-4869-8368-6DEBA77B919F/kagglecatsanddogs_5340.zip")

train_ds, validation_ds, test_ds = tfds.load(
    "cats_vs_dogs",
    # Reserve 10% for validation and 10% for test
    split=["train[:40%]", "train[40%:50%]", "train[50%:60%]"],
    as_supervised=True,  # Include labels
)

print("Number of training samples: %d" % tf.data.experimental.cardinality(train_ds))
print(
    "Number of validation samples: %d" % tf.data.experimental.cardinality(validation_ds)
)
print("Number of test samples: %d" % tf.data.experimental.cardinality(test_ds))

Dataset cats_vs_dogs downloaded and prepared to /root/tensorflow_datasets/cats_vs_dogs/4.0.0. Subsequent calls will reuse this data.
Number of training samples: 9305
Number of validation samples: 2326
Number of test samples: 2326


## TODO1: 데이터셋 전처리하기

In [3]:
# 각 데이터셋의 크기 변경하기
size = (150, 150)

train_ds = train_ds.map(lambda x, y: (tf.image.resize(x, size), y))
validation_ds = validation_ds.map(lambda x, y: (tf.image.resize(x, size), y))
test_ds = test_ds.map(lambda x, y: (tf.image.resize(x, size), y))

In [4]:
# Xception의 전처리 기법 적용하기
from tensorflow.keras.applications.xception import Xception
from tensorflow.keras.applications.xception import preprocess_input

train_ds = train_ds.map(lambda x, y: (preprocess_input(tf.image.resize(x, size)), y))
validation_ds = validation_ds.map(lambda x, y: (preprocess_input(tf.image.resize(x, size)), y))
test_ds = test_ds.map(lambda x, y: (preprocess_input(tf.image.resize(x, size)), y))


In [5]:
# 데이터셋을 32장씩 묶읍시다.
batch_size = 32

train_ds = train_ds.cache().batch(batch_size).prefetch(buffer_size=10)
validation_ds = validation_ds.cache().batch(batch_size).prefetch(buffer_size=10)
test_ds = test_ds.cache().batch(batch_size).prefetch(buffer_size=10)

## TODO2: 데이터 증강 층 만들기<br>


In [11]:
from tensorflow import keras
from tensorflow.keras import layers

data_augmentation = keras.Sequential(
    [
        layers.experimental.preprocessing.RandomFlip("horizontal"),
        layers.experimental.preprocessing.RandomRotation(0.1),
        layers.experimental.preprocessing.RandomZoom(0.1),
    ]
)


## TODO3: 모델 정의하기

가져온 모델 앞에 입력층과 데이터 층강층을 추가하고 뒤에 예측하는 부분 (MLP와 같은 Dense층) 을 추가합니다.

In [14]:
from keras import models
from keras import layers

base_model = keras.applications.Xception(
    weights="imagenet", # ImageNet에 대해서 훈련된 모델을 가져옵니다.
    input_shape=(150, 150, 3),
    include_top=False,  # ImageNet 데이터를 예측하는 출력층은 포함하지 않습니다.
)  
# Xception은 훈련이 불가능하도록 설정합니다.
base_model.trainable = False

model = models.Sequential()

#TODO: 입력층 추가
model.add(layers.Input(shape=(150, 150, 3)))

#TODO: 데이터 증강층 추가
model.add(data_augmentation)
#가져온 모델 추가
model.add(base_model)

# MLP는 일차원으로 펼쳐진 입력이 필요합니다. 
# 아래 둘 중 원하는 것 하나만 쓰세요.
# Xception은 5 x 5 사이즈의 2048개 채널을 출력합니다.
#model.add(layers.Flatten()) # 각 채널의 모든 픽셀들을 일차원으로 이어붙이는 무식하고 낡은 방법 (25개 x 2048장)
#model.add(layers.GlobalAveragePooling2D()) # 각 5 x 5 채널을 하나의 값으로 요약한 후 일차원으로 이어붙이는 방법 (2048개)

# 출력층 추가 (0~1 출력하는 노드 하나만 필요하고 적절한 활성화 함수 기재)
model.add(layers.GlobalAveragePooling2D())
model.add(layers.Dense(1, activation="sigmoid"))
# 컴파일
model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"]) 

In [15]:
# 직접 만들었던 모델과 비교해보세요.
model.summary()

Model: "sequential_8"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 sequential_5 (Sequential)   (None, 150, 150, 3)       0         
                                                                 
 xception (Functional)       (None, 5, 5, 2048)        20861480  
                                                                 
 global_average_pooling2d (G  (None, 2048)             0         
 lobalAveragePooling2D)                                          
                                                                 
 dense (Dense)               (None, 1)                 2049      
                                                                 
Total params: 20,863,529
Trainable params: 2,049
Non-trainable params: 20,861,480
_________________________________________________________________


## TODO4: 훈련하기

In [16]:
from keras.callbacks import EarlyStopping

# TODO: 자유롭게 설정하세요.
epochs = 10000
es = EarlyStopping(patience=3, restore_best_weights=True)
model.fit(train_ds, epochs=epochs, callbacks=es, validation_data=validation_ds)

Epoch 1/10000
291/291 [==============================] - 40s 85ms/step - loss: 0.1723 - accuracy: 0.9280 - val_loss: 0.0789 - val_accuracy: 0.9721
Epoch 2/10000
291/291 [==============================] - 22s 74ms/step - loss: 0.1261 - accuracy: 0.9444 - val_loss: 0.0771 - val_accuracy: 0.9695
Epoch 3/10000
291/291 [==============================] - 22s 76ms/step - loss: 0.1167 - accuracy: 0.9493 - val_loss: 0.0740 - val_accuracy: 0.9699
Epoch 4/10000
291/291 [==============================] - 22s 75ms/step - loss: 0.1159 - accuracy: 0.9541 - val_loss: 0.0803 - val_accuracy: 0.9686
Epoch 5/10000
291/291 [==============================] - 22s 76ms/step - loss: 0.1038 - accuracy: 0.9556 - val_loss: 0.0725 - val_accuracy: 0.9703
Epoch 6/10000
291/291 [==============================] - 22s 76ms/step - loss: 0.1026 - accuracy: 0.9591 - val_loss: 0.0705 - val_accuracy: 0.9729
Epoch 7/10000
291/291 [==============================] - 23s 79ms/step - loss: 0.1048 - accuracy: 0.9586 - val_loss: 0

## 테스트 데이터로 평가해보기

In [17]:
model.evaluate(test_ds)

73/73 [==============================] - 4s 58ms/step - loss: 0.0705 - accuracy: 0.9738


[0.07049636542797089, 0.9737747311592102]

<img src='https://img1.daumcdn.net/thumb/R1280x0/?scode=mtistory2&fname=https%3A%2F%2Fblog.kakaocdn.net%2Fdn%2FW0TLL%2FbtqRx0uGeC2%2FKxbOgpwhzXXvwu1VtcmjNK%2Fimg.jpg' height=200>

## 세부 튜닝하기

가져온 모델인 Xception까지 우리 데이터에 맞게 훈련해봅시다.<br>
너무 많은 시간이 소요될 수 있으니 적당히 수행합니다. (중간에 중단 버튼 눌러도 괜찮음)

In [19]:
# 가져온 모델 훈련 가능하도록 설정
base_model.trainable = True

# 훈련 가능한 파라미터 수를 위와 비교해보세요.
model.summary()

# 컴파일 해주세요.
optimizer = keras.optimizers.Adam(lr=1e-5)
model.compile(loss="binary_crossentropy", optimizer=optimizer, metrics=["accuracy"])
# 방금 훈련한 모델을 조금 더 훈련시킵니다.
epochs = 100
model.fit(train_ds, epochs=epochs, callbacks=es, validation_data=validation_ds)

Model: "sequential_8"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 sequential_5 (Sequential)   (None, 150, 150, 3)       0         
                                                                 
 xception (Functional)       (None, 5, 5, 2048)        20861480  
                                                                 
 global_average_pooling2d (G  (None, 2048)             0         
 lobalAveragePooling2D)                                          
                                                                 
 dense (Dense)               (None, 1)                 2049      
                                                                 
Total params: 20,863,529
Trainable params: 20,809,001
Non-trainable params: 54,528
_________________________________________________________________


Epoch 1/100
291/291 [==============================] - 103s 234ms/step - loss: 0.2608 - accuracy: 0.9005 - val_loss: 0.3142 - val_accuracy: 0.9003
Epoch 2/100
291/291 [==============================] - 68s 235ms/step - loss: 0.1460 - accuracy: 0.9411 - val_loss: 0.1241 - val_accuracy: 0.9549
Epoch 3/100
291/291 [==============================] - 69s 238ms/step - loss: 0.1069 - accuracy: 0.9592 - val_loss: 0.1667 - val_accuracy: 0.9458
Epoch 4/100
291/291 [==============================] - 69s 239ms/step - loss: 0.0983 - accuracy: 0.9608 - val_loss: 0.1192 - val_accuracy: 0.9579
Epoch 5/100
291/291 [==============================] - 70s 240ms/step - loss: 0.0841 - accuracy: 0.9689 - val_loss: 0.0974 - val_accuracy: 0.9613
Epoch 6/100
291/291 [==============================] - 69s 238ms/step - loss: 0.0856 - accuracy: 0.9687 - val_loss: 0.1794 - val_accuracy: 0.9488
Epoch 7/100
291/291 [==============================] - 69s 239ms/step - loss: 0.0727 - accuracy: 0.9737 - val_loss: 0.0883 

## 다시 테스트 데이터로 평가해보기

In [20]:
model.evaluate(test_ds)

73/73 [==============================] - 5s 63ms/step - loss: 0.0994 - accuracy: 0.9652


[0.09942273795604706, 0.9651762843132019]

적은 시간을 사용했을 경우 오히려 성능이 떨어질 가능성이 높습니다.<br>
원본 노트북에서는 훈련 불가능하게 만든 상태에서 20 epochs, 세부 튜닝에서 10 epochs 훈련하는데 성능이 조금 좋아집니다.<br>
전이학습에서 세부 튜닝하는 방법까지 공부한 것에 의의를 둡시다.